In [1]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(1, os.path.join(sys.path[0], "../00_helpers"))
from helpers import get_rice_index, get_splitted_share

In [2]:
BOLD = '\033[1m'
CLEAR = '\033[0m'

In [3]:
df = pd.read_csv("../processed_data/votes_finaux.csv")
dfp = pd.read_csv("../processed_data/parlementaires_with_age.csv")
dfp.rename(columns={"Unnamed: 0.1.1": "id"}, inplace=True)

### Select political parties with at least 5 councillors

In [4]:
party_count = pd.DataFrame(dfp["party"].value_counts())
party_count.columns = ["count"]
selected_parties = party_count[party_count["count"] > 4]

In [5]:
# ! Doublons parmi les votes finaux
df[df.AffairTitle.duplicated()].shape

(49, 318)

In [6]:
# En cas de doublon, pour le moment, on ne garde que le dernier vote
df = df.sort_values("VoteDate").drop_duplicates(
    subset="AffairTitle", keep="last"
).set_index("AffairTitle").copy()
len(df)

303

## Get votes with largest young/old gap

In [7]:
party_delta = {}
for party_name in selected_parties.index:
    councillor_ids_a = [
        str(i) for i in dfp[(dfp["party"] == party_name) & (dfp["Age"] >= 40)]["id"]
    ]
    councillor_ids_b = [
        str(i) for i in dfp[(dfp["party"] == party_name) & (dfp["Age"] < 40)]["id"]
    ]

    print(
        f"\n{BOLD}{party_name}{CLEAR}:\n {len(councillor_ids_a)} >= 40\n {len(councillor_ids_b)} < 40"
    )

    if len(councillor_ids_b) > 0:
        df_votes_a = df[councillor_ids_a].T.copy()
        df_votes_b = df[councillor_ids_b].T.copy()

        votes = []
        for col in df_votes_b.columns:
            len_votes = len(df_votes_b[df_votes_b[col].isin(["Oui", "Non"])])
            if len_votes == 0:
                # print("No yes/no vote for", col)
                pass
            else:
                vote_share = get_splitted_share(df_votes_a, df_votes_b, vote_column=col)
                vote_share["date"] = df.T[col]["VoteDate"]
                vote_share["object"] = col
                votes.append(vote_share)
        dfv = pd.DataFrame(votes)
        print(f"\nTop 2 objects with largest gap:")
        dfv.sort_values("delta", ascending=False).head(5).apply(
            lambda row: print(
                f"Delta: {round(row['delta'], 1)} percent points, %yes <40: {round(row['share b'])}% ({row['yes b']}/{row['yes/no b']}), %yes >=40: {round(row['share a'])}%. Object: {row['object']} ({row['date']})"
            ),
            axis=1,
        )
        party_delta[party_name] = dfv
    else:
        print("No councillor < 40 => no comparison")


UDC:
 75 >= 40
 5 < 40

Top 2 objects with largest gap:
Delta: 86.4 percent points, %yes <40: 0% (0/1), %yes >=40: 86%. Object: Pour sauver des vies en favorisant le don d'organes. Initiative populaire. Loi sur la transplantation. Modification (2021-10-01 08:47:10)
Delta: 75.6 percent points, %yes <40: 100% (4/4), %yes >=40: 24%. Object: Stop à l’îlot de cherté – pour des prix équitables. Initiative populaire et contre-projet indirect (2021-03-19 09:20:13)
Delta: 57.4 percent points, %yes <40: 100% (2/2), %yes >=40: 43%. Object: Transport souterrain de marchandises. Loi (2021-12-17 08:32:28)
Delta: 52.2 percent points, %yes <40: 100% (3/3), %yes >=40: 48%. Object: Reprise et mise en oeuvre du règlement (UE) 2019/1896 du Parlement européen et du Conseil relatif au corps européen de garde-frontières et de garde-côtes et abrogeant les règlements (UE) no 1052/2013 et (UE) 2016/16 24, avec une modification de la loi sur l'asile (2021-10-01 08:44:01)
Delta: 42.3 percent points, %yes <40: 50

In [8]:
# Results for 6 political parties with enough <40 y.o. councillors
party_delta.keys()

dict_keys(['UDC', 'PSS', 'PLR', 'VERT-E-S', 'M-E', 'pvl'])

In [10]:
# Example usage
party_delta['PSS'].sort_values('delta', ascending=False).head(20).round(1)

,yes a,yes b,yes/no a,yes/no b,share a,share b,delta,date,object
13,28,1,32,3,87.5,33.3,54.2,2016-03-18 09:12:25,Loi sur la surveillance de la correspondance p...
144,32,2,32,4,100.0,50.0,50.0,2018-09-28 10:47:10,Projet fiscal 17
155,11,0,29,9,37.9,0.0,37.9,2020-05-06 13:07:51,Révision partielle urgente de la loi fédérale ...
113,34,2,35,3,97.1,66.7,30.5,2017-12-15 09:34:43,Développement de l'acquis de Schengen. Reprise...
262,28,6,28,8,100.0,75.0,25.0,2021-10-01 08:49:49,Adaptation de l'âge limite en vigueur au sein ...
4,11,2,25,3,44.0,66.7,22.7,2015-12-18 08:18:29,Pour un revenu de base inconditionnel. Initiat...
120,11,1,20,3,55.0,33.3,21.7,2018-03-16 08:26:45,Pour la souveraineté alimentaire. L’agricultur...
87,32,3,34,4,94.1,75.0,19.1,2017-06-16 08:46:16,Ancrer durablement le taux spécial de TVA appl...
111,22,2,27,2,81.5,100.0,18.5,2017-12-15 09:32:36,Pour une monnaie à l’abri des crises: émission...
259,18,4,22,4,81.8,100.0,18.2,2021-10-01 08:47:10,Pour sauver des vies en favorisant le don d'or...
